In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
import pylab as plt
from scipy import stats
import xlwings as xw

/Users/matthias/anaconda/lib/python2.7/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory
  stacklevel=stacklevel + 1):


In [23]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-11-03/"
#path = "/Users/matthias/Desktop/PhD Temp/2017-02-22/"
#path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 2463
files_end = 2470

In [24]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["116", "117", "118", "119", "120", "122", "124"]
cup_config = cycle_Sb_2

# Mass Range of cup configuration
mass_range = cycle_Sb_2_mass_range

# Isotopes used for Interference correction
#corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111", "Te" : "126", "Xe": "129"}
#corr_isotopes_2 = {}
corr_isotopes_Sb = {"Te": "125", "Xe": "129"}

norm_ratio = ["123","121"]
denom_isotope = "118"
Sn_monitor = ["125"]


def eval_iso_list(isotopes_list,denom_isotope, monitor_iso):
    isotope_nom = norm_ratio[0]
    isotope_den = denom_isotope
    isotopes_list.remove(isotope_den)
    for i in range(len(monitor_iso)):
        isotopes_list.append(monitor_iso[i])
    isotopes_list.sort()
    return isotopes_list

isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope, Sn_monitor)

data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [isotope_ls]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

In [25]:
isotope_ls

['116', '117', '119', '120', '122', '124', '125']

In [26]:
database["Xe"]["Ratios"].get_all_ratios("129")

{'124': 0.0035974268469331047,
 '126': 0.003354211852574769,
 '128': 0.07257966445566791,
 '130': 0.15393354073312387,
 '131': 0.8025725373327319,
 '132': 1.0170204315988616,
 '134': 0.39447778815973994,
 '136': 0.33498555333720215}

In [27]:
mass_range.get_graph_of_corr(corr_isotopes_Sb)

{'116': set(),
 '117': set(),
 '118': set(),
 '119': set(),
 '120': {'125'},
 '121': set(),
 '122': {'125'},
 '123': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '127': set(),
 '129': set()}

In [28]:
mass_range.get_mass_range()

{'116': ['Sn', 'Cd'],
 '117': ['Sn'],
 '118': ['Sn'],
 '119': ['Sn'],
 '120': ['Sn', 'Te'],
 '121': ['Sb'],
 '122': ['Sn', 'Te'],
 '123': ['Sb', 'Te'],
 '124': ['Sn', 'Te', 'Xe'],
 '125': ['Te'],
 '127': ['I'],
 '129': ['Xe']}

In [29]:
mass_range.get_order_of_corr(corr_isotopes_Sb)

[{'116', '117', '118', '119', '121', '125', '127', '129'},
 {'120', '122', '123', '124'}]

In [30]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(5552,5623,1) + range(5632,5729, 1) 

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_all = df_all.append(df_zero_1, ignore_index = True)
    #df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2015-11-03 12:56:00,1,blank sol,0.011494,0.014348,2.240920e-05,0.000387,0.001923,0.000294,0.002556,0.002381,0.000493,0.001378,0.000434,0.000845,0.000083
2015-11-03 12:56:00,2,blank sol,0.011487,0.014386,-6.521500e-06,0.000279,0.001779,0.000165,0.002266,0.001753,0.000366,0.000931,0.000287,0.000573,0.000070
2015-11-03 12:56:00,3,blank sol,0.011501,0.014371,-4.086200e-06,0.000234,0.001606,0.000138,0.002095,0.001484,0.000281,0.000728,0.000227,0.000464,0.000054
2015-11-03 12:56:00,4,blank sol,0.011461,0.014369,4.527400e-06,0.000175,0.001519,0.000079,0.001960,0.001235,0.000167,0.000519,0.000175,0.000327,0.000092
2015-11-03 12:56:00,5,blank sol,0.011411,0.014310,-1.773230e-05,0.000209,0.001494,0.000079,0.001943,0.001196,0.000199,0.000522,0.000170,0.000361,0.000040
2015-11-03 12:56:00,6,blank sol,0.011487,0.014362,1.426940e-05,0.000215,0.001669,0.000124,0.002172,0.001575,0.000276,0.000761,0.000224,0.000465,0.000056
2015-11-03 12:56:00,7,blank sol,0.011472,0.014352,2.543250e-05,0.000149,0.001455,0.000089,0.001908,0.001140,0.000167,0.000417,0.000142,0.000280,0.000065
2015-11-03 12:56:00,8,blank sol,0.011487,0.014355,3.324560e-05,0.000169,0.001441,0.000076,0.001905,0.001130,0.000176,0.000466,0.000150,0.000285,0.000057
2015-11-03 12:56:00,9,blank sol,0.011490,0.014301,1.891630e-05,0.000249,0.001459,0.000134,0.001937,0.001521,0.000306,0.000739,0.000242,0.000455,0.000053


In [31]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb, denom_corr_ratio)
        #new_corr.line2_corr(df_zero, "119")
        #df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        #df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        #new_corr.line2_corr(df_zero, "119")
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.external_norm_Sb(norm_ratio, denom_isotope, iter_beta)
    #data_sample = new_corr.raw_ratios_corr(denom_isotope)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_internal_norm = df_internal_norm.append(data_sample)

In [32]:
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

#df_internal_sub["Cd/Sn"] = (df_internal_norm["111"]/12.722)/(df_internal_norm["118"]/24.223)
df_internal_sub["Te/Sn"] = (df_internal_norm["125"]/7.0509)/(df_internal_norm["118"]/24.223)

#df_internal_sub["Te/Sn"] = (df_internal_norm["126"]/18.8066)/(df_internal_norm["118"]/24.223)
#df_internal_sub = df_internal_sub[df_internal_sub["118"] >= 7]
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")
df_internal_sub

,cycle,sample,116/118,117/118,119/118,120/118,122/118,124/118,125/118,116,117,118,119,120,122,124,125,Te/Sn
date,,,,,,,,,,,,,,,,,,
2015-11-03 13:03:00,1,NIST 100ppb,0.600434,0.316753,0.354538,1.345546,0.191110,0.238762,2.883484e-07,4.217450,2.255279,7.216234,2.592819,9.971082,1.453698,1.863456,2.279170e-06,1.085049e-06
2015-11-03 13:03:00,2,NIST 100ppb,0.600435,0.316772,0.354528,1.345430,0.191102,0.238715,-3.457961e-06,4.237614,2.266396,7.252015,2.605835,10.021413,1.461337,1.873278,-2.748453e-05,-1.302006e-05
2015-11-03 13:03:00,3,NIST 100ppb,0.600435,0.316739,0.354557,1.345601,0.191123,0.238780,-1.733826e-06,4.243614,2.269232,7.261366,2.609242,10.034355,1.463028,1.875520,-1.379263e-05,-6.525475e-06
2015-11-03 13:03:00,4,NIST 100ppb,0.600432,0.316775,0.354547,1.345458,0.191101,0.238744,-1.330844e-06,4.228857,2.261606,7.236160,2.600114,9.998482,1.457788,1.868743,-1.055023e-05,-5.008840e-06
2015-11-03 13:03:00,5,NIST 100ppb,0.600445,0.316739,0.354554,1.345510,0.191098,0.238745,-2.863313e-06,4.241731,2.268264,7.258545,2.608300,10.030490,1.462473,1.874906,-2.277443e-05,-1.077907e-05
2015-11-03 13:03:00,6,NIST 100ppb,0.600411,0.316753,0.354532,1.345540,0.191113,0.238745,7.368503e-08,4.222732,2.258373,7.226673,2.596728,9.986981,1.456263,1.866860,5.835700e-07,2.774201e-07
2015-11-03 13:03:00,7,NIST 100ppb,0.600465,0.316745,0.354562,1.345561,0.191097,0.238751,-3.478782e-06,4.210981,2.251750,7.205415,2.589221,9.957088,1.451653,1.861030,-2.746383e-05,-1.309440e-05
2015-11-03 13:03:00,8,NIST 100ppb,0.600457,0.316777,0.354538,1.345470,0.191107,0.238738,7.992373e-07,4.186015,2.238612,7.162506,2.573576,9.896725,1.442979,1.849644,6.271270e-06,3.007973e-06
2015-11-03 13:03:00,9,NIST 100ppb,0.600473,0.316752,0.354545,1.345595,0.191109,0.238765,-9.454006e-07,4.162599,2.225843,7.122215,2.559144,9.841929,1.434858,1.839419,-7.376330e-06,-3.558022e-06


In [33]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_internal_norm_mean = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).mean()
df_internal_norm_mean_sd = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).agg([np.mean, SD2])
df_internal_norm_mean["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
df_internal_norm_mean_sd["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
#df_internal_norm_mean_sd = df_internal_norm_mean_sd[1:8]
#df_internal_norm_mean = df_internal_norm_mean[df_internal_norm_mean["118"] >= 7]
df_internal_norm_mean_sd
#epsilon = epsilon.drop(["118"], axis = 1)
#plotplot = df_internal_norm_mean[(df_internal_norm_mean["125"] < 0.00010)  & (df_internal_norm_mean["124/120"] > 0.17745)] 
#plot(plotplot["125"], plotplot["124/120"], "ro")

116/118             117/118             119/118  \
                         mean       SD2      mean       SD2      mean   
date                                                                    
2015-11-03 13:03:00  0.600428  0.000054  0.316754  0.000027  0.354550   
2015-11-03 13:12:00  0.600438  0.000047  0.316758  0.000023  0.354552   
2015-11-03 13:20:00  0.600450  0.000085  0.316762  0.000028  0.354560   
2015-11-03 13:29:00  0.600430  0.000053  0.316756  0.000019  0.354560   
2015-11-03 13:38:00  0.600443  0.000047  0.316762  0.000022  0.354564   

                                120/118             122/118            \
                          SD2      mean       SD2      mean       SD2   
date                                                                    
2015-11-03 13:03:00  0.000022  1.345503  0.000154  0.191114  0.000028   
2015-11-03 13:12:00  0.000022  1.345518  0.000103  0.191120  0.000027   
2015-11-03 13:20:00  0.000025  1.345511  0.000130  0.191117  0.000021   
2015-11-03 13:29:00  0.000024  1.345541  0.000129  0.191123  0.000026   
2015-11-03 13:38:00  0.000023  1.345535  0.000124  0.191126  0.000020   

                        ...            120       122                 124  \
                        ...            SD2      mean       SD2      mean   
date                    ...                                                
2015-11-03 13:03:00     ...       0.135352  1.455085  0.019847  1.865297   
2015-11-03 13:12:00     ...       0.190296  1.405070  0.027765  1.801297   
2015-11-03 13:20:00     ...       0.337793  1.428618  0.049343  1.831568   
2015-11-03 13:29:00     ...       0.119538  1.435879  0.017473  1.840933   
2015-11-03 13:38:00     ...       0.172732  1.396666  0.025203  1.790650   

                                    125               Te/Sn            \
                          SD2      mean       SD2      mean       SD2   
date                                                                    
2015-11-03 13:03:00  0.025481 -0.000014  0.000022 -0.000006  0.000010   
2015-11-03 13:12:00  0.035589 -0.000013  0.000025 -0.000006  0.000013   
2015-11-03 13:20:00  0.063240 -0.000028  0.000022 -0.000014  0.000010   
2015-11-03 13:29:00  0.022420 -0.000006  0.000017 -0.000003  0.000008   
2015-11-03 13:38:00  0.032385 -0.000018  0.000017 -0.000009  0.000008   

                          sample  
                                  
date                              
2015-11-03 13:03:00  NIST 100ppb  
2015-11-03 13:12:00  NIST 100ppb  
2015-11-03 13:20:00  NIST 100ppb  
2015-11-03 13:29:00  NIST 100ppb  
2015-11-03 13:38:00  NIST 100ppb  

[5 rows x 33 columns]

In [19]:
df_internal_norm_sample_sd = df_internal_norm_mean.groupby(df_internal_norm_mean["sample"]).agg([np.mean, SD2])
df_internal_norm_sample_sd["124/118"]

,mean,SD2
sample,,
BHVO-2 1_2(2) 1,0.238853,0.000000
BHVO-2 1_2(2) 2,0.238854,0.000000
BHVO-2 1_2(2) 3,0.238866,0.000000
BHVO-2 2_1,0.238932,0.000000
Cd-Sn-Std 1 1,0.238911,0.000000
Cd-Sn-Std 1 2,0.247557,0.017939
DTS-2B 1_1 1,0.238864,0.000000
DTS-2B 1_1 2,0.238878,0.000000
DTS-2B 1_1 3,0.238884,0.000000


In [66]:
#df_internal_norm_mean.describe()

In [ ]:
# variation of standard data + mean + 2SD
sel_ratio = "124/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].median(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].quantile(0.25), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].quantile(0.75), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# outlier corr variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = new_corr.mad_outlier_rejection(plot_Nist.drop("sample", axis=1))
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [20]:
std_used = "NIST 100ppb"
sub_index = df_internal_norm_mean[(df_internal_norm_mean["sample"].shift(1) == df_internal_norm_mean["sample"].shift(-1)) & (df_internal_norm_mean["sample"].shift(1) == std_used)].index
#corr_values = df_internal_norm_mean[(df_external_norm_mean >= 0.74289)]
#corr_values = df_internal_norm_mean[:"20150110 10:00"]
corr_values = df_internal_norm_mean[df_internal_norm_mean["118"] >= 0.1]
corr_values_2 = corr_values[corr_values["sample"] == std_used]
#corr_values = df_internal_norm_mean
sample = corr_values["sample"]
sample_2 = corr_values_2["sample"]
std_mean = corr_values[(corr_values["sample"] == std_used)].mean()
new = corr_values[corr_values.columns[corr_values.dtypes != 'object']]
new_2 = corr_values_2[corr_values_2.columns[corr_values_2.dtypes != 'object']]

#std_mean["112/120", "114/120", "115/120", "117/120", "118/120", "119/120", "122/120", "124/120"] = [0.029823, 0.020189, 0.010372, 0.235299, 0.742923, 0.263443, 0.142078, 0.177546]
#print std_mean
#epsilon =  ((new / std_mean)-1)*10000
#new["sample"] = sample
epsilon_NIST =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*1000
epsilon_2 = ((new_2 / ((new_2.shift(1) + new_2.shift(-1))/2))-1)*1000
epsilon_NIST["sample"] = sample
epsilon_2["sample"] = sample

epsilon_NIST = epsilon_NIST[new.index.isin(sub_index)]
epsilon_NIST= epsilon_NIST[epsilon_NIST["sample"] != std_used]
epsilon_2 = epsilon_2[epsilon_2["sample"] == std_used]
epsilon_comp = epsilon_NIST.append(epsilon_2, ignore_index=False)
#epsilon_comp.index = epsilon_comp["Date"]
#epsilon_comp.columns
epsilon_comp = epsilon_comp.sort_index(axis=0)

In [21]:
#epsilon_comp = epsilon_comp.drop([datetime.strptime("2017-02-24 15:18", "%Y-%m-%d %H:%M")])
epsilon_comp

,116/118,117/118,119/118,120/118,122/118,124/118,125/118,116,117,118,119,120,122,124,125,Te/Sn,sample
date,,,,,,,,,,,,,,,,,
2015-11-03 12:48:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIST 100ppb
2015-11-03 13:03:00,0.005763,0.014166,0.006609,-0.009087,-0.002148,-0.004294,-4886.606140,34.293216,34.310405,34.305474,34.317471,34.325246,34.339204,34.357916,-4941.498319,-3311.617760,NIST 100ppb
2015-11-03 13:12:00,-0.002000,-0.000446,-0.006596,0.009418,0.022739,0.021672,-586.191050,-25.554620,-25.549160,-25.541328,-25.536544,-25.530881,-25.503826,-25.493047,-603.753952,-640.949099,NIST 100ppb
2015-11-03 13:20:00,0.027234,0.014582,0.011077,-0.013876,-0.021033,-0.040064,14513.714537,5.693649,5.696807,5.695485,5.719450,5.720771,5.731570,5.738401,15649.919426,15630.801024,NIST 100ppb
2015-11-03 13:29:00,-0.028352,-0.016797,-0.007443,0.012300,0.007616,0.044317,-1180.248322,16.474718,16.474490,16.479548,16.457286,16.467199,16.445449,16.462212,-1180.785887,-1130.011016,NIST 100ppb
2015-11-03 13:38:00,0.015416,0.013556,0.005043,-0.003919,0.008585,-0.030149,-21917.624650,-16.646497,-16.640476,-16.640966,-16.620727,-16.624693,-16.587190,-16.605047,-20147.626224,-36470.071370,NIST 100ppb
2015-11-03 13:53:00,-0.016212,-0.019714,0.010250,0.010452,0.017496,0.040819,-805.543760,2.656805,2.627405,2.618025,2.599260,2.569406,2.520845,2.489504,-808.248623,-840.595854,NIST 100ppb
2015-11-03 14:02:00,0.014101,0.017820,-0.015400,-0.009190,-0.028784,-0.032624,168.379677,-10.144679,-10.106584,-10.094753,-10.080557,-10.042037,-9.999440,-9.944086,151.220053,142.160681,NIST 100ppb
2015-11-03 14:10:00,-0.013128,-0.009733,0.007099,-0.003309,0.014770,0.007429,1800.662612,24.252221,24.233499,24.231335,24.225362,24.207000,24.189538,24.157763,1863.814128,2102.593439,NIST 100ppb


In [24]:
def SD2(x):
    SD2 = np.nanstd(x, ddof=1) * 2
    return SD2

epsilon = epsilon_comp.drop(["125", "116", "117", "118", "119", "120", "122", "124", "Te/Sn"], axis = 1)
#epsilon = epsilon.drop(["118"], axis = 1)
epsilon_sum = np.round(epsilon.groupby("sample").agg([np.mean, SD2]), decimals=3)
epsilon_sum

116/118        117/118        119/118        120/118  \
                            mean    SD2    mean    SD2    mean    SD2    mean   
sample                                                                          
BHVO-2 1_2(2) 1           -0.142    NaN  -0.091    NaN   0.046    NaN   0.134   
BHVO-2 1_2(2) 2           -0.146    NaN  -0.082    NaN   0.069    NaN   0.146   
BHVO-2 1_2(2) 3           -0.154    NaN  -0.094    NaN   0.045    NaN   0.144   
BHVO-2 2_1                -0.265    NaN  -0.129    NaN   0.127    NaN   0.269   
DTS-2B 1_1 1              -0.126    NaN  -0.096    NaN   0.081    NaN   0.145   
DTS-2B 1_1 2              -0.210    NaN  -0.112    NaN   0.109    NaN   0.177   
DTS-2B 1_1 3              -0.183    NaN  -0.109    NaN   0.098    NaN   0.152   
J2 1_1                    -0.198    NaN  -0.121    NaN   0.082    NaN   0.182   
J2 1_1 2                  -0.198    NaN  -0.110    NaN   0.071    NaN   0.159   
J2 2_1(1) 1               -0.133    NaN  -0.089    NaN   0.084    NaN   0.159   
J2 2_1(1) 2               -0.173    NaN  -0.106    NaN   0.080    NaN   0.176   
J2 2_1(1) 3               -0.141    NaN  -0.076    NaN   0.090    NaN   0.171   
NIST 100ppb                0.000  0.029   0.000  0.026   0.000  0.026  -0.000   
NIST 100ppb + 0.005ppb U   0.015  0.005   0.010  0.005   0.022  0.046  -0.017   
NIST 100ppb + 0.03ppb U    0.043    NaN   0.017    NaN   0.118    NaN  -0.055   
NIST 2_1 1                 0.004    NaN   0.001    NaN   0.012    NaN  -0.003   
NIST 2_1 2                 0.024    NaN   0.004    NaN   0.001    NaN  -0.002   
NIST 2_1 3                 0.034    NaN   0.008    NaN   0.001    NaN  -0.009   
NIST Ag1x8 1               0.039    NaN   0.023    NaN  -0.013    NaN  -0.012   
NIST Ag1x8 2               0.004    NaN   0.009    NaN  -0.002    NaN  -0.041   
NIST Ag1x8 3               0.010    NaN  -0.014    NaN  -0.019    NaN  -0.046   
NIST EV HCl 3              0.033    NaN   0.020    NaN  -0.014    NaN  -0.041   
NIST EV HCl 3 2            0.041    NaN   0.027    NaN  -0.042    NaN  -0.076   
NIST EV HCl 6              0.024    NaN   0.006    NaN  -0.016    NaN  -0.028   
NIST EV HCl 6 2            0.039    NaN   0.017    NaN   0.001    NaN  -0.022   
NIST EV HCl 7              0.005    NaN   0.008    NaN   0.004    NaN  -0.024   
NIST EV HCl 7 2            0.024    NaN  -0.002    NaN  -0.029    NaN  -0.095   
NIST EV NO3-HF 3           0.061    NaN   0.024    NaN  -0.043    NaN  -0.086   
NIST EV NO3-HF 3 2         0.061    NaN   0.033    NaN  -0.026    NaN  -0.074   
SPEX 100ppb               -0.163  0.028  -0.089  0.028   0.075  0.017   0.156   
ZH09-05 10.3(1) 1         -0.531    NaN  -0.294    NaN   0.239    NaN   0.500   
ZH09-05 10.3(1) 2         -0.527    NaN  -0.291    NaN   0.242    NaN   0.523   
ZH09-05 10.3(1) 3         -0.528    NaN  -0.286    NaN   0.245    NaN   0.507   
ZH09-05 23.3(1) 1         -1.092    NaN  -0.670    NaN   0.641    NaN   1.289   
ZH09-05 23.3(1) 2         -1.125    NaN  -0.685    NaN   0.649    NaN   1.300   
ZH09-05 23.3(1) 3         -1.134    NaN  -0.679    NaN   0.641    NaN   1.296   

                                122/118        124/118           125/118  \
                            SD2    mean    SD2    mean    SD2       mean   
sample                                                                     
BHVO-2 1_2(2) 1             NaN   0.268    NaN   0.382    NaN  -9076.835   
BHVO-2 1_2(2) 2             NaN   0.282    NaN   0.393    NaN   4155.389   
BHVO-2 1_2(2) 3             NaN   0.276    NaN   0.423    NaN  -1390.401   
BHVO-2 2_1                  NaN   0.512    NaN   0.756    NaN    697.289   
DTS-2B 1_1 1                NaN   0.272    NaN   0.383    NaN -35640.796   
DTS-2B 1_1 2                NaN   0.352    NaN   0.540    NaN   1383.751   
DTS-2B 1_1 3                NaN   0.336    NaN   0.514    NaN  -5296.268   
J2 1_1                      NaN   0.368    NaN   0.546    NaN -11443.648   
J2 1_1 2                    N

In [100]:
### sel_ratio = "112/118"
std = "NIST 100ppb"

def plot_epsilon(sel_ratio, std, xlim):
    x = range(1, len(epsilon_sum.index)+1)
    plt.errorbar(epsilon_sum[sel_ratio]["mean"], x, xerr = epsilon_sum[sel_ratio]["SD2"], ls='none', marker="o")
    plt.title("samples" + ", 2SD = " + str(np.round((2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std()),2)) + ", n= " +  str((epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].count())))
    plt.xlim(xlim)
    plt.ylim([0.5, len(epsilon_sum.index)+1])
    plt.yticks(x, epsilon_sum.index, size='small', rotation=0)
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean(), color='k', linestyle='-')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()+ 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()- 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    #plt.axvline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
    plt.ylabel(r'$\mathregular{\epsilon^{'+sel_ratio+'}}$Sn$_{NIST}$')
    #savefig(path+"epsilon_118.pdf", bbox_inches='tight')

In [ ]:
wb = xw.books.add()
#wb.sheets.add()
wb.sheets["Sheet1"].range('A1').value = df_all
df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
wb.sheets["Sheet1"].range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[3].range('AF1').value = df_internal_sub
#df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
#df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
#xw.Range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[4].range('A21').value = df_internal_norm_mean_sd

In [ ]:
wb.sheets[4].range('BF21').value = epsilon_comp

In [ ]:
#epsilon_comp[epsilon_comp["sample"]== "NIST 200ppb"]["117/118"].std()
wb.sheets[4].range('BF36').value = epsilon_sum

In [ ]:
#sel_ratio = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
#sel_ratio = ["116/117", "118/117", "119/117", "120/117", "122/117", "124/117"]
sel_ratio = ["116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]
#sel_ratio = ["112/120", "114/120", "115/120"]
#sel_ratio = ["112/118", "114/118", "115/118"]
xlim = [-1, 1]

fig = plt.figure(figsize=(10,15))

for i in range(len(sel_ratio)):
    ax = plt.subplot(5,2,i+1)
    plot_epsilon(sel_ratio[i], "NIST 200ppb", xlim)
fig.tight_layout()
#wb.sheets[4].pictures.add(fig, name= '116/120 norm - bgd_corr 1', update = True)
#savefig(path+"epsilon_x_bgd_118_2_100ppb.pdf", bbox_inches='tight')

In [ ]:
sel_ratio = "116/118"
epsilon_comp = epsilon_comp[epsilon_comp["sample"] == "NIST 200ppb"]
x = epsilon_comp.index
plt.plot(x, epsilon_comp[sel_ratio], marker="o")
plt.ylim([-0.5, 0.5])
#plt.xticks(x, epsilon_comp["sample"], size='small', rotation=90)
plt.axhline(epsilon_comp[sel_ratio].mean(), color='k', linestyle='-')
plt.axhline(epsilon_comp[sel_ratio].mean()+ 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(epsilon_comp[sel_ratio].mean()- 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.ylabel(r'$\mathregular{\epsilon^{118}}$Sn')
#savefig(path+"epsilon_119.pdf", bbox_inches='tight')

In [76]:
df_all.to_csv(path + "Sn_H8_L4_2_cycles_NU_Signals_zero_corr.csv", mode='w', header=True)

In [99]:
df_internal_norm.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_122_118_bgd_c3orr_outlier_corr_all_Xe_new_126Te.csv", mode='w', header=True)

In [100]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_122_118_bgd_corr_outlier_corr_Xe_new_126Te.csv", mode='w', header=True)

In [ ]:
df_std.to_csv(path_sav + "All_standards_internal_norm_mean.csv")

In [ ]:
df_all_mean_1 = pd.read_csv(path_sav + "Sn_H9_L4_1_cycles_internal_norm_mean.csv")

In [ ]:
df_all_mean_2 = pd.read_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm_mean.csv")

In [ ]:
df_mean_all = df_all_mean_1.append(df_all_mean_2)

In [ ]:
#old Cd/Sn & Te/Sn extraction - doesn't include bgd corr
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "SQ") & (df_all["sample"] != "teflon blank")]
df_internal_sub["111"] = df_all_sub["L3 (1)"]
df_internal_sub["125"] = df_all_sub["H5 (2)"]
df_internal_sub["Cd/Sn"] = (df_all_sub["L3 (1)"]/12.722)/(df_all_sub["H4 (1)"]/24.223)
df_internal_sub["Te/Sn"] = (df_all_sub["H5 (2)"]/7.0509)/(df_all_sub["H4 (1)"]/24.223)
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")

In [ ]:
df_mean_all = df_mean_all[2:]
df_mean_all.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
df_mean_all = df_mean_all[["Date", "112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118","sample"]]
df_mean_all = df_mean_all.set_index("Date")

In [ ]:
df_std = df_mean_all[(df_mean_all["sample"] == "SPEX1 100ppb") | (df_mean_all["sample"] == "SPEX 100ppb") | (df_mean_all["sample"] == "NIST 100ppb")]
df_std

In [ ]:
df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]] = df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]].astype(float)
df_std.index = df_std.index.astype(datetime64)
df_std = df_std[(df_std.index < pd.Timestamp('2015-03-10')) | (df_std.index > pd.Timestamp('2015-03-12'))]
df_std = df_std[(df_std.index < pd.Timestamp('2014-11-29 18:25:00')) | (df_std.index > pd.Timestamp('2014-11-29 20:27:00'))]
df_std

In [ ]:
df_std = df_std[(df_std["118"] > 5) & (df_std["118/120"] > 0.7428)]
for i in df_std.index[df_std["sample"] == "SPEX1 100ppb"]:
    df_std.ix[i, "sample"] = "SPEX 100ppb"

In [ ]:
sample = df_std["sample"]
std_mean = df_std[(df_std["sample"] == "SPEX 100ppb")].mean()
new = df_std[df_std.columns[df_std.dtypes != 'object']]
epsilon =  ((new / std_mean)-1)*10000
epsilon["sample"] = sample
epsilon
epsilon[(epsilon["118/120"] > 0.4) | (epsilon["118/120"] < -0.4)]

In [ ]:
lee = pd.DataFrame({"112/120" : pd.Series(0.029812), "114/120" : pd.Series(0.020195), "115/120": pd.Series(0.010366), "116/120": pd.Series(0.4460), "117/120" : pd.Series(0.235313), "118/120" : pd.Series(0.742935), "119/120" : pd.Series(0.263430), "122/120" : pd.Series(0.142086), "124/120" : pd.Series(0.177588)})
epsilon_lee = ((lee / std_mean[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]])-1)*10000
epsilon_lee["sample"] = "Lee et al. (1995)"
epsilon_lee